In [ ]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

- 데이터프레임 준비작업

In [ ]:
from google.colab import files # 컴퓨터 내에 존재하는 "부동산-11월25일 수정.csv"파일을 코랩의 저장공간으로 불러들인다.
myfile = files.upload()

Saving 부동산-11월25일 수정.csv to 부동산-11월25일 수정.csv


In [ ]:
df = pd.read_csv('/content/부동산-11월25일 수정.csv') # 코랩의 저장공간에 존재하는 csv파일을 불러들인다.
df 

,Room,Size,Traffic,Parking,School,Area,Dealing
0,three room,medium,no,no,no,Gyesu_dong,low
1,three room,low,no,yes,no,Gogang_dong,low
2,three room,medium,no,no,yes,Gogang_dong,low
3,three room,medium,no,no,yes,Gogang_dong,low
4,three room,medium,yes,no,no,Gogang_dong,low
...,...,...,...,...,...,...,...
568,apartment,high,yes,no,no,Chunui_dong,medium
569,apartment,high,yes,no,no,Chunui_dong,medium
570,apartment,high,yes,no,no,Chunui_dong,high
571,apartment,high,yes,no,no,Chunui_dong,high


- Entropy 구하기

In [ ]:
from functools import reduce

def getEntropy(df, feature) :
    D_len = df[feature].count() # 데이터 전체 길이
    # reduce함수를 이용하여 초기값 0에 
    # 각 feature별 count을 엔트로피 식에 대입한 값을 순차적으로 더함
    return reduce(lambda x, y: x+(-(y[1]/D_len) * np.log2(y[1]/D_len)), \
                  df[feature].value_counts().items(), 0)

In [ ]:
getEntropy(df, "Dealing") # Entropy값 출력

1.5633281730776094

- Gain 구하기

In [ ]:
def NoNan(x):#Nan의 경우 0을 반환
    return np.nan_to_num(x)

def getGainA(df, feature) :
    info_D = getEntropy(df, feature) # 목표변수 Feature에 대한 Info(Entropy)를 구한다.
    columns = list(df.loc[:, df.columns != feature]) # 목표변수를 제외한 나머지 설명변수들을 리스트 형태로 저장한다.
    gains = []
    D_len = df.shape[0] # 전체 길이
    for col in columns:
        info_A = 0
        # Col내 개별 attribute_value 이름(c_name)과 attribute_value별 갯수(c_len)
        for c_name, c_len in df[col].value_counts().items():

            try:
              target_low = df.groupby([col,feature]).size()[c_name]['low'] # 해당 attribute에서 각각의 attribute_value 중에서 class가 low인 것의 갯수를 센다.
            except:
              target_low=0
        
            try:
              target_medium = df.groupby([col,feature]).size()[c_name]['medium'] # 해당 attribute에서 각각의 attribute_value 중에서 class가 medium인 것의 갯수를 센다.
            except:
              target_medium=0
         
            try:
              target_high = df.groupby([col,feature]).size()[c_name]['high'] # 해당 attribute에서 각각의 attribute_value 중에서 class가 high인 것의 갯수를 센다.
            except:
              target_high=0


            prob_l = target_low / c_len # low일 때의 발생 확률을 구한다.
            prob_m = target_medium / c_len # medium일 때의 발생 확률을 구한다
            prob_h =target_high / c_len # high일 때의 발생 확률을 구한다

            # Info_A(Remainder) <- |Dj|/|D| *  Entropy(label) 
            info_A += (c_len/D_len) * -(NoNan(prob_l*np.log2(prob_l)) + NoNan(prob_m*np.log2(prob_m)) + NoNan(prob_h*np.log2(prob_h)))
          
        gains.append(info_D - info_A) # Gain을 구한다. 
    
    result = dict(zip(columns,gains)) # 각 변수에 대한 Information Gain 을 Dictionary 형태로 저장한다.
    return(result)

In [ ]:
getGainA(df, "Dealing") # Gain값 출력

{'Area': 0.601781265121514,
 'Parking': 0.2032714314735642,
 'Room': 0.43961363251401187,
 'School': 0.001739585513650388,
 'Size': 0.49405651800413786,
 'Traffic': 0.1392563719653399}